In [117]:
import sympy
import numpy as np
import matplotlib.pyplot as plt
import dill
dill.settings["recurse"] = True
from tqdm.notebook import tqdm
import plotly

from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve
import gmsh

In [118]:
weak_form_functional_lambdified = dill.load(open("../calculations/macro_cubic_rhct_weak_form_functional_xxyy", "rb"))
weak_form_right_part_lambdified = dill.load(open("../calculations/macro_cubic_rhct_weak_form_right_part", "rb"))
interpolation_function =          dill.load(open("../calculations/macro_cubic_rhct_basis", "rb"))
mapping_function_lambdified =     dill.load(open("../calculations/macro_cubic_rhct_mapping_function", "rb"))

## Get mesh

In [154]:
gmsh.initialize()
gmsh.model.add("tri_mesh")

# Define points
point_mesh_size = 0.9
point_1 = gmsh.model.geo.addPoint(0, 0, 0, point_mesh_size)
point_2 = gmsh.model.geo.addPoint(1, 0, 0, point_mesh_size)
point_3 = gmsh.model.geo.addPoint(0, 1, 0, point_mesh_size)
point_4 = gmsh.model.geo.addPoint(-1, 0, 0, point_mesh_size)
point_5 = gmsh.model.geo.addPoint(0, -1, 0, point_mesh_size)

# Define circles
arc_1 = gmsh.model.geo.addCircleArc(point_2, point_1, point_3)
arc_2 = gmsh.model.geo.addCircleArc(point_3, point_1, point_4)
arc_3 = gmsh.model.geo.addCircleArc(point_4, point_1, point_5)
arc_4 = gmsh.model.geo.addCircleArc(point_5, point_1, point_2)

# Define line loop
circle = gmsh.model.geo.addCurveLoop([arc_1, arc_2, arc_3, arc_4])

# Define plane surface
plane = gmsh.model.geo.addPlaneSurface([circle])

gmsh.model.geo.synchronize()
# gmsh.option.setNumber(name="Mesh.Smoothing", value=30)


# Make regular triangulation -->>
gmsh.model.mesh.setTransfiniteCurve(arc_1, 411)
gmsh.model.mesh.setTransfiniteCurve(arc_2, 411)
gmsh.model.mesh.setTransfiniteCurve(arc_3, 411)
gmsh.model.mesh.setTransfiniteCurve(arc_4, 411)
gmsh.model.mesh.setTransfiniteSurface(plane)
# gmsh.model.mesh.setRecombine(2, ps1)

physical_group_curves_tag = gmsh.model.addPhysicalGroup(1, [arc_1, arc_2, arc_3, arc_4], name="Boundary curves")
physical_group_surface_tag = gmsh.model.addPhysicalGroup(2, [plane], name="Surface")




point_tag = point_1  # replace with your point's tag

# Define the Attractor field
attractor_field = gmsh.model.mesh.field.add("Attractor")
gmsh.model.mesh.field.setNumbers(attractor_field, "NodesList", [point_tag])

# Define the Threshold field
threshold_field = gmsh.model.mesh.field.add("Threshold")
gmsh.model.mesh.field.setNumber(threshold_field, "IField", attractor_field)
gmsh.model.mesh.field.setNumber(threshold_field, "LcMin", 0.01)
gmsh.model.mesh.field.setNumber(threshold_field, "LcMax", 0.5)
gmsh.model.mesh.field.setNumber(threshold_field, "DistMin", 0.1)
gmsh.model.mesh.field.setNumber(threshold_field, "DistMax", 0.6)

# Set the Threshold field as the background field
gmsh.model.mesh.field.setAsBackgroundMesh(threshold_field)


gmsh.model.mesh.generate(2)
# gmsh.fltk.run()

In [155]:
boundary_node_tags, boundary_node_coords = gmsh.model.mesh.getNodesForPhysicalGroup(1, physical_group_curves_tag)
surface_node_tags, points = gmsh.model.mesh.getNodesForPhysicalGroup(2, physical_group_surface_tag)
points = points.reshape(-1, 3)[:, :-1]
boundary_node_tags = boundary_node_tags - 1

In [156]:
element_types, element_tags, element_node_tags = gmsh.model.mesh.getElements()
triangles = [nodes for elem_type, nodes in zip(element_types, element_node_tags) if elem_type == 2][0]
triangles = triangles.reshape(-1, 3).astype(int)
triangles = triangles - 1

In [157]:
vertex_marker_is_boundary = np.zeros(surface_node_tags.shape[0])
vertex_marker_is_boundary[boundary_node_tags] = 1

In [158]:
total_points = points.shape[0]
NUM = 3*total_points

In [159]:
x, y = sympy.symbols("x y")

In [160]:
F = sympy.Float(1)
F_x = F.diff(x)
F_y = F.diff(y)

In [161]:
right_part_values = np.zeros((total_points, 3))
right_part_values[:, 0] = sympy.lambdify((x, y), F)(*points.T)
right_part_values[:, 1] = sympy.lambdify((x, y), F_x)(*points.T)
right_part_values[:, 2] = sympy.lambdify((x, y), F_y)(*points.T)

In [162]:
def orient_batch(arg):
    indices = np.argsort(arg, axis=-1)
    vertices = np.take_along_axis(arg, indices, axis=-1)

    return vertices

In [163]:
triangles = orient_batch(triangles)

In [164]:
batched_triangles = points[triangles]

In [165]:
bilinear_form = weak_form_functional_lambdified(batched_triangles[:, 0, 0],
                                                batched_triangles[:, 0, 1],
                                                batched_triangles[:, 1, 0],
                                                batched_triangles[:, 1, 1],
                                                batched_triangles[:, 2, 0],
                                                batched_triangles[:, 2, 1],
                                    )

righ_hand_side = weak_form_right_part_lambdified(batched_triangles[:, 0, 0],
                                        batched_triangles[:, 0, 1],
                                        batched_triangles[:, 1, 0],
                                        batched_triangles[:, 1, 1],
                                        batched_triangles[:, 2, 0],
                                        batched_triangles[:, 2, 1],
                                        right_part_values[triangles[:, 0], 0],
                                        right_part_values[triangles[:, 0], 1],
                                        right_part_values[triangles[:, 0], 2],
                                        right_part_values[triangles[:, 1], 0],
                                        right_part_values[triangles[:, 1], 1],
                                        right_part_values[triangles[:, 1], 2],
                                        right_part_values[triangles[:, 2], 0],
                                        right_part_values[triangles[:, 2], 1],
                                        right_part_values[triangles[:, 2], 2],                                        
                                        ).squeeze()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Kamo\anaconda3\envs\thesis\lib\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Kamo\AppData\Local\Temp\ipykernel_8692\3369627148.py", line 1, in <module>
    bilinear_form = weak_form_functional_lambdified(batched_triangles[:, 0, 0],
  File "C:\Users\Kamo\AppData\Local\Temp\ipykernel_10856\2188328864.py", line 3, in weak_form_functional_xxyy_lambdified_quadrature
  File "<lambdifygenerated-2>", line 165, in _lambdifygenerated
numpy.core._exceptions._ArrayMemoryError: Unable to allocate 64.1 MiB for an array with shape (25, 336200) and data type float64

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Kamo\anaconda3\envs\thesis\lib\site-packages\IPython\core\interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Us

In [147]:
matrix = lil_matrix((NUM, NUM))
b = np.zeros(NUM)

for tidx, element in enumerate(tqdm(triangles)):
    
    x0, x1, x2 = points[element[0], 0], points[element[1], 0], points[element[2], 0]
    y0, y1, y2 = points[element[0], 1], points[element[1], 1], points[element[2], 1]
    
    wff = bilinear_form[tidx]
    wfrp = righ_hand_side[tidx]
    # wff = weak_form_functional_lambdified(x0, y0, x1, y1, x2, y2)
    
    # wfrp = weak_form_right_part_lambdified(
    #                                         x0, y0, x1, y1, x2, y2,
                                            
    #                                         right_part_values[element[0], 0],
    #                                         right_part_values[element[0], 1],
    #                                         right_part_values[element[0], 2],
                                            
    #                                         right_part_values[element[1], 0],
    #                                         right_part_values[element[1], 1],
    #                                         right_part_values[element[1], 2],
                                            
    #                                         right_part_values[element[2], 0],
    #                                         right_part_values[element[2], 1],
    #                                         right_part_values[element[2], 2],                                            
                                                                                                                                  
    #                                        )
    
    for trial_vertex_idx in range(3):
        
        if vertex_marker_is_boundary[element[trial_vertex_idx]] == 1:
            
            for trial_dof_idx in range(3):      
                
                row_index = 3*element[trial_vertex_idx] + trial_dof_idx
                col_index = 3*element[trial_vertex_idx] + trial_dof_idx
                matrix[row_index, col_index] = 1
                         
                b[row_index] = 0
                    
        else:   
            for trial_dof_idx in range(3):
                row_index = 3*element[trial_vertex_idx] + trial_dof_idx
                row_index_f = 3*trial_vertex_idx + trial_dof_idx
                
                for test_vertex_idx in range(3):
                    for test_dof_idx in range(3):
                        
                        col_index = 3*element[test_vertex_idx] + test_dof_idx                        
                        col_index_f = 3*test_vertex_idx + test_dof_idx
                        
                        matrix[row_index, col_index] += wff[row_index_f, col_index_f]
                        
                b[row_index] += wfrp[row_index_f]                                                                                    

  0%|          | 0/200 [00:00<?, ?it/s]

In [148]:
matrix_csc = matrix.tocsc()

In [149]:
c = spsolve(matrix_csc, b)

In [150]:
F =  c[0 : (3 * total_points) : 3]
Fx = c[1 : (3 * total_points) : 3]
Fy = c[2 : (3 * total_points) : 3]

In [151]:
param = np.linspace(0, 1, 10)
gridx, gridy = np.meshgrid(param, param, indexing="ij")

mask = (gridy <= 1 - gridx)

gridx = gridx[mask]
gridy = gridy[mask]

In [152]:
global_x, global_y, global_z = [], [], []

for element in triangles:
    
    x0, x1, x2 = points[element[0], 0], points[element[1], 0], points[element[2], 0]
    y0, y1, y2 = points[element[0], 1], points[element[1], 1], points[element[2], 1]
    
    physical_x, physical_y = mapping_function_lambdified(gridx, gridy, x0, y0, x1, y1, x2, y2).squeeze()
    
    physical_z = interpolation_function(
                                        gridx, gridy,
                                        
                                        x0, y0, 
                                        x1, y1, 
                                        x2, y2,
                                        
                                        F[element[0]], 
                                        Fx[element[0]], 
                                        Fy[element[0]],
                                        
                                        F[element[1]], 
                                        Fx[element[1]], 
                                        Fy[element[1]],
                                        
                                        F[element[2]], 
                                        Fx[element[2]], 
                                        Fy[element[2]],
                                        "f"
                                        ).squeeze()
    
    global_x.append(physical_x)
    global_y.append(physical_y)
    global_z.append(physical_z)
    
global_x = np.concatenate(global_x)
global_y = np.concatenate(global_y)
global_z = np.concatenate(global_z)

# Plot

In [153]:
plot_data = [
    plotly.graph_objects.Mesh3d(
                                x=global_x, 
                                y=global_y,
                                z=global_z,
                                intensity=global_z
                                )
    ]
fig = plotly.graph_objects.Figure(data=plot_data)
fig.update_layout()

# Elementwise plot

In [ ]:
plot_data = [
    plotly.graph_objects.Mesh3d(
                                x=gx, 
                                y=gy,
                                z=gz, 
                                )
        for gx, gy, gz in zip(
                                global_x.reshape(-1, gridx.shape[0]), 
                                global_y.reshape(-1, gridx.shape[0]), 
                                global_z.reshape(-1, gridx.shape[0])
                              )
    ]
fig = plotly.graph_objects.Figure(data=plot_data)
fig.update_layout()